In [1]:
import torch
import numpy as np
import torchvision.models as models
import torch.nn as nn
model = models.vgg11(pretrained=True)


In [2]:
from PIL import Image
import torchvision.transforms as transforms

In [3]:
# Set PIL to be tolerant of image files that are truncated.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [5]:
def process_image(np_im):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    np_im=np_im.reshape(240,240,3)
    transformations = transforms.Compose([transforms.ToTensor()])
    torch_image = transformations(np_im).float()    
    return torch_image

In [6]:
def predict(np_im):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to 
    predicted ImageNet class for image at specified path
    
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to VGG-16 model's prediction
    '''

    
    if use_cuda:
        model.cuda()
    
    model.eval()    
    
    with torch.no_grad():
       
        image = process_image(np_im)
        if use_cuda:
            image = image.type(torch.FloatTensor).cuda()   
        #I do not understand why we shold do this why not 3 dimensions are not sufficent.
        #print(image.shape)
        image = image.unsqueeze(0)
        #print(image.shape)
        
        output = model.forward(image)
        output =output.cpu().detach().numpy()
        #ps = torch.exp(logps)
        #_,cls_idx=torch.max(output, 1)
        #probs_tensor, classes_tensor = ps.topk(1,dim=1)

    #return probs_tensor, classes_tensor    
    #return cls_idx.item()
    
    return output

In [23]:
for param in model.features.parameters():
    param.requires_grad = False
    
for cparam in model.classifier.parameters():
    cparam.requires_grad = False    
#number thrusters on the Lilly   
thruster=4
first_layer= nn.Linear(25088, 2048)
model.classifier[0] = first_layer
# new layers automatically have requires_grad = True
second_layer= nn.Linear(2048, 1024)
model.classifier[3] = second_layer
last_layer = nn.Linear(1024, thruster)
model.classifier[6] = last_layer

In [24]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print("cudaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
print(use_cuda)
# move model to GPU if CUDA is available
if use_cuda:
    model = model.cuda()

cudaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
True


In [25]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [26]:
model.classifier[3].weight[0].size()

torch.Size([2048])

In [33]:
model.classifier[3].weight[333]

tensor([-0.0026, -0.0016, -0.0096,  ...,  0.0031,  0.0107, -0.0020],
       device='cuda:0', grad_fn=<SelectBackward>)

In [31]:
model.features[3].weight[4]

tensor([[[ 2.9998e-02, -9.6848e-03,  2.3259e-02],
         [-1.6198e-02, -2.5749e-02, -1.5478e-02],
         [-2.8136e-02, -2.5018e-02,  5.1302e-03]],

        [[ 1.0161e-01, -5.1544e-03,  5.9574e-02],
         [ 6.5373e-02, -3.5419e-02,  1.3825e-01],
         [-3.4036e-02,  4.0542e-02,  6.4423e-02]],

        [[-6.8146e-03, -6.5456e-02, -4.4173e-02],
         [-6.0832e-02, -1.2182e-01, -1.0191e-01],
         [-8.4842e-02, -1.2506e-01, -6.0528e-02]],

        [[ 1.2993e-02, -9.7796e-03, -1.3994e-02],
         [ 1.6030e-02,  1.1112e-02, -1.3064e-02],
         [-3.4138e-02,  2.2668e-02, -5.0603e-02]],

        [[ 2.1709e-02,  4.0281e-03,  1.3583e-02],
         [-2.2482e-02, -9.6295e-03,  5.7518e-03],
         [-2.2874e-02, -1.9240e-02,  1.2477e-03]],

        [[ 6.4129e-03, -1.1395e-01, -7.9008e-02],
         [-3.3199e-02, -1.4287e-01, -6.1077e-02],
         [-4.3069e-02, -7.5073e-02, -2.8882e-02]],

        [[-6.1784e-03,  1.5382e-03,  1.4033e-02],
         [ 1.3265e-03, -3.2954e-02,  2

In [16]:
print(model.hidden.weight[0])

AttributeError: 'VGG' object has no attribute 'hidden'

In [18]:
for param in model.parameters():
    print(param,param.size())

Parameter containing:
tensor([[ 0.0060,  0.0188,  0.0346,  ..., -0.0294, -0.0120,  0.0054],
        [-0.0194,  0.0344, -0.0279,  ..., -0.0071, -0.0074, -0.0324],
        [-0.0264,  0.0038,  0.0271,  ...,  0.0048, -0.0325,  0.0220],
        ...,
        [-0.0356,  0.0344, -0.0288,  ...,  0.0252,  0.0243, -0.0307],
        [ 0.0062, -0.0042, -0.0140,  ...,  0.0175,  0.0325,  0.0059],
        [-0.0092,  0.0150, -0.0075,  ...,  0.0162,  0.0247,  0.0012]],
       requires_grad=True) torch.Size([256, 784])
Parameter containing:
tensor([-0.0121, -0.0149,  0.0135, -0.0033, -0.0219, -0.0047,  0.0140,  0.0191,
         0.0054, -0.0284, -0.0082, -0.0229, -0.0201, -0.0233,  0.0101, -0.0329,
         0.0132, -0.0218,  0.0117, -0.0169, -0.0047, -0.0035,  0.0121,  0.0196,
        -0.0271, -0.0186,  0.0111,  0.0248,  0.0161,  0.0150, -0.0066, -0.0313,
        -0.0086, -0.0078,  0.0353,  0.0084, -0.0024, -0.0317,  0.0217, -0.0001,
        -0.0064, -0.0008, -0.0109, -0.0329,  0.0075,  0.0104,  0.0260, -

In [15]:
model.hidden.weight

Parameter containing:
tensor([[ 0.0060,  0.0188,  0.0346,  ..., -0.0294, -0.0120,  0.0054],
        [-0.0194,  0.0344, -0.0279,  ..., -0.0071, -0.0074, -0.0324],
        [-0.0264,  0.0038,  0.0271,  ...,  0.0048, -0.0325,  0.0220],
        ...,
        [-0.0356,  0.0344, -0.0288,  ...,  0.0252,  0.0243, -0.0307],
        [ 0.0062, -0.0042, -0.0140,  ...,  0.0175,  0.0325,  0.0059],
        [-0.0092,  0.0150, -0.0075,  ...,  0.0162,  0.0247,  0.0012]],
       requires_grad=True)

In [21]:
model.hidden.weight.data

tensor([[ 0.0060,  0.0188,  0.0346,  ..., -0.0294, -0.0120,  0.0054],
        [-0.0194,  0.0344, -0.0279,  ..., -0.0071, -0.0074, -0.0324],
        [-0.0264,  0.0038,  0.0271,  ...,  0.0048, -0.0325,  0.0220],
        ...,
        [-0.0356,  0.0344, -0.0288,  ...,  0.0252,  0.0243, -0.0307],
        [ 0.0062, -0.0042, -0.0140,  ...,  0.0175,  0.0325,  0.0059],
        [-0.0092,  0.0150, -0.0075,  ...,  0.0162,  0.0247,  0.0012]])

In [16]:
model.hidden.bias

Parameter containing:
tensor([-0.0121, -0.0149,  0.0135, -0.0033, -0.0219, -0.0047,  0.0140,  0.0191,
         0.0054, -0.0284, -0.0082, -0.0229, -0.0201, -0.0233,  0.0101, -0.0329,
         0.0132, -0.0218,  0.0117, -0.0169, -0.0047, -0.0035,  0.0121,  0.0196,
        -0.0271, -0.0186,  0.0111,  0.0248,  0.0161,  0.0150, -0.0066, -0.0313,
        -0.0086, -0.0078,  0.0353,  0.0084, -0.0024, -0.0317,  0.0217, -0.0001,
        -0.0064, -0.0008, -0.0109, -0.0329,  0.0075,  0.0104,  0.0260, -0.0310,
        -0.0105, -0.0170, -0.0343, -0.0069,  0.0212,  0.0124,  0.0123,  0.0229,
        -0.0165, -0.0290,  0.0150, -0.0038,  0.0196, -0.0097, -0.0327, -0.0345,
         0.0079, -0.0115, -0.0192, -0.0265,  0.0108,  0.0160,  0.0134,  0.0262,
         0.0062,  0.0189,  0.0059, -0.0295, -0.0074, -0.0251,  0.0091,  0.0182,
         0.0141,  0.0293, -0.0345, -0.0129,  0.0255, -0.0068,  0.0256,  0.0199,
        -0.0049, -0.0208, -0.0321, -0.0018,  0.0243, -0.0052, -0.0183,  0.0206,
         0.0323, -